<a href="https://colab.research.google.com/github/arjasc5231/moodots/blob/ACRNN/SER/CNN/CRNNdeep_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import sys
sys.path.append("/content/drive/MyDrive/team_runner/colab/emoDB/CNN")
from kfold import kfold

In [ ]:
X_train, X_test, Y_train, Y_test = np.load("/content/drive/MyDrive/team_runner/colab/dataset/emoDB/emo_mel_3d.npy", allow_pickle=True) # 0.8265+-0.0140
#X_train, X_test, Y_train, Y_test = np.load("/content/drive/MyDrive/team_runner/colab/dataset/emoDB/emo_mel_more.npy", allow_pickle=True) # 정확도 0.7942+-0.0276
X = np.concatenate((X_train, X_test), axis=0)
#X = np.expand_dims(X, axis=-1)
Y = np.concatenate((Y_train, Y_test), axis=0)

print(X.shape)
print(Y.shape)

(911, 128, 128, 3)
(911,)


In [ ]:
def create_model():
    inputs = keras.Input(shape=(128, 128, 3))
    # inputs = keras.Input(shape=(128, 128, 1))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)(inputs)
    pool1 = keras.layers.MaxPool2D()(conv1)
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)(pool1)
    pool2 = keras.layers.MaxPool2D()(conv2)
    conv3 = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)(pool2)
    pool3 = keras.layers.MaxPool2D()(conv3)

    trans = keras.layers.Permute((2,1,3))(pool3)     # 확실히 시간순으로 lstm에 학습을 안하면 overfitting만 된다. 정확도 50%
    reshape = keras.layers.Reshape((-1, 16*128))(trans) # 열 개수(freq축)*ch
    linear = keras.layers.Dense(512)(reshape)
    lstm = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=False, dropout=0.5))(linear)

    fc = keras.layers.Dense(128)(lstm)
    drop = keras.layers.Dropout(0.5)(fc) # 없어도 77%는 나오네
    output = keras.layers.Dense(7)(drop)
    return keras.Model(inputs=inputs, outputs=output)

In [ ]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0     

In [ ]:
kfold(5, create_model, X, Y, 0.001, 100, 120, 10)
# 특히 3D데이터를 쓰고 loss가 매우 작은 수에서 논다. lr decay가 필요할듯

Learning started. k=5
<<fold 1>>
Epoch	loss		train acc	test acc
10  	0.7028  	0.7739  	0.6393
20  	0.2592  	0.9275  	0.6834
30  	0.0486  	0.9937  	0.7926
40  	0.0058  	1.0000  	0.7876
50  	0.0026  	1.0000  	0.7986
60  	0.0017  	1.0000  	0.7876
70  	0.0010  	1.0000  	0.8086
80  	0.0006  	1.0000  	0.7976
90  	0.0006  	1.0000  	0.8046
100  	0.0012  	1.0000  	0.8046
110  	0.0004  	1.0000  	0.8146
120  	0.0004  	1.0000  	0.8086
max train accuracy: 1.0000
max test accuracy: 0.8207
<<fold 2>>
Epoch	loss		train acc	test acc
10  	0.7538  	0.7322  	0.6765
20  	0.2530  	0.9350  	0.6571
30  	0.0243  	0.9987  	0.7730
40  	0.0489  	0.9950  	0.7204
50  	0.0036  	1.0000  	0.7880
60  	0.0010  	1.0000  	0.7598
70  	0.0006  	1.0000  	0.7809
80  	0.0005  	1.0000  	0.7780
90  	0.0007  	1.0000  	0.7820
100  	0.0005  	1.0000  	0.7891
110  	0.0004  	1.0000  	0.7952
120  	0.0005  	1.0000  	0.7870
max train accuracy: 1.0000
max test accuracy: 0.8174
<<fold 3>>
Epoch	loss		train acc	test acc
10  	0.7340  	0.7528